In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv('C:/Users/Administrator/Desktop/pythonHistoricalTesting/data/SCData/YMH21-CBOT.scid_BarData_used4test.txt',parse_dates = {'Time' : [0,1]})
test.columns = ['Time','Open','High','Low','Close','Volume','NumberOfTrades','BidVolume','AskVolume']

In [3]:
#若不转化成float后面会报错
test.Open = [float(x) for x in test.Open]
test.High = [float(x) for x in test.High]
test.Low  = [float(x) for x in test.Low]
test.Close  = [float(x) for x in test.Close]
test.Volume  = [float(x) for x in test.Volume]

In [4]:
from features.Bars import *
from features.Technics import *
test = IdentifyRsi(test)
test = IdentifyPinBar(test)
test = IdentifyTopBottomType(test)
test = IdentifyPregnantType(test)
test = IdentifyTriplePregnantType(test)
test = IdentifySwallowType(test)

In [5]:
UpPregnant = []
[UpPregnant.append(1) if (test.loc[i].UpPregnantType == 1) | (test.loc[i].UpTriplePregnantType == 1) else UpPregnant.append(0) for i in range(len(test))]
test['UpPregnant'] = UpPregnant

In [6]:
DownPregnant = []
[DownPregnant.append(1) if (test.loc[i].DownPregnantType == 1) | (test.loc[i].DownTriplePregnantType == 1) else DownPregnant.append(0) for i in range(len(test))]
test['DownPregnant'] = DownPregnant

In [7]:
Analysis_deputy = test.copy()
for i in range(len(Analysis_deputy)):
    if Analysis_deputy.DownPregnant.loc[i] == 1:
        test.DownPregnant.loc[(i + 1) : (i + 3)] = 1
        i += 4

D:\anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [8]:
for i in range(len(Analysis_deputy)):
    if Analysis_deputy.UpPregnant.loc[i] == 1:
        test.UpPregnant.loc[(i + 1) :(i + 3)] = 1
        i += 4

In [9]:
for i in range(len(Analysis_deputy)):
    if Analysis_deputy.UpSwallowType.loc[i] == 1:
        test.UpSwallowType.loc[(i + 1) :(i + 2)] = 1
        i += 3

In [10]:
for i in range(len(Analysis_deputy)):
    if Analysis_deputy.DownSwallowType.loc[i] == 1:
        test.DownSwallowType.loc[(i + 1) :(i + 2)] = 1
        i += 3

In [11]:
for i in range(len(Analysis_deputy)):
    if Analysis_deputy.TopType.loc[i] == 1:
        test.TopType.loc[i + 1] = 1
        i += 2

In [12]:
for i in range(len(Analysis_deputy)):
    if Analysis_deputy.BottomType.loc[i] == 1:
        test.BottomType.loc[i + 1] = 1
        i += 2

In [13]:
#往df中添加二次平滑的RSI
def AddRSI_T(df,T):
    '''
    由于RSI14是在index = 14时才有，故rsi2应该在index = 15开始才有意义，rsi7应该在index = 20时才有意义
    '''
    RSI_T = [0]* (T - 1 + 14)
    for i in range((T - 1 + 14),len(df)):
        RSI_T_sub = df[(i - T + 1) : (i + 1)].RSI.mean()
        RSI_T.append(RSI_T_sub)
    df['RSI_{}'.format(T)] = RSI_T

In [14]:
AddRSI_T(test,2)
AddRSI_T(test,7)

In [15]:
#给Analysis_Data增加previousRSI
test['previousRSI'] = test.RSI.shift(1)

In [16]:
def RSI_SMA_shortgodown(df,T):
    '''
        df为包含二次平滑完毕的RSI的表
        T为需要研究的区间长度，只要在T时间内有上穿或者下穿就认为有穿
    '''
        
    RSIShortgodown = []
    for i in range(len(df)):
        RSI_short = df.iloc[i:i + T +1,22].tolist()###这边要注意18是短期的平滑的rsi所在的位置，如果df表格不一样，这里需要修改
        RSI_long = df.iloc[i:i + T +1,23].tolist()
    
        #diff = RSI2 - RSI7#存在由正变负的就ok
        diff = [RSI_short[i] - RSI_long[i] for i in range(len(RSI_short))]
        #print(diff)
        for j in range(len(diff) - 1):
            #print(j)
            before_minus_after = diff[j] - diff[j + 1]
            rsi_short = RSI_short[j]
            rsi_long = RSI_long[j]
            #if (before_minus_after >= diff[j]) & (diff[j] >= 0) & (rsi_short >= 50) & (rsi_short <= 100)& (rsi_long >= 50) & (rsi_long <= 100):给rsi限定范围
            if (before_minus_after >= diff[j]) & (diff[j] >= 0):
                RSIShortgodown_sub = 1
                break
            else:
                RSIShortgodown_sub = 0
                continue
        RSIShortgodown.append(RSIShortgodown_sub)
    #print(len(RSIShortgodown))
    df['RSIShortgodown'] = RSIShortgodown

In [17]:
RSI_SMA_shortgodown(test,3)

In [18]:
def RSI_SMA_shortgoup(df,T):
    '''
    df为包含二次平滑完毕的RSI的表
    T为需要研究的区间长度，只要在T时间内有上穿或者下穿就认为有穿
    '''
    RSIShortgoup = []
    for i in range(len(df)):
        RSI_short = df.iloc[i:i + T +1,22].tolist()###这边要注意18是短期的平滑的rsi所在的位置，如果df表格不一样，这里需要修改
        RSI_long = df.iloc[i:i + T +1,23].tolist()
        
        #diff = RSI2 - RSI7#存在由正变负的就ok\n",
        diff = [RSI_long[i] - RSI_short[i] for i in range(len(RSI_short))]
        #print(diff)\n",
        for j in range(len(diff) - 1):
            #print(j)\n",
            before_minus_after = diff[j] - diff[j + 1]
            rsi_short = RSI_short[j]
            rsi_long = RSI_long[j]
            #if (before_minus_after >= diff[j]) & (diff[j] >= 0) & (rsi_short >= 50) & (rsi_short <= 100)& (rsi_long >= 50) & (rsi_long <= 100):
            if (before_minus_after >= diff[j]) & (diff[j] >= 0):
                RSIShortgoup_sub = 1
                break
            else:
                RSIShortgoup_sub = 0
                continue
        RSIShortgoup.append(RSIShortgoup_sub)
    #print(len(RSIShortgodown))\n",
    df['RSIShortgoup'] = RSIShortgoup

In [19]:
RSI_SMA_shortgoup(test,3)

In [20]:
#标记出极值点的index
import features.zigzag as fz
zig = fz.ZigZag(test,0.005)#为什么换了数据设置的百分比有差别
df_his_listed = zig.fixed_zzps(add_once=True)#历史被擦除的也显示了
df_only_last = zig.fixed_zzps()

In [21]:
df_his_listed['High_or_Low'] = [str(df_his_listed.type_.loc[i]) for i in range(len(df_his_listed))]
df_only_last['High_or_Low'] = [str(df_only_last.type_.loc[i]) for i in range(len(df_only_last))]

In [22]:
BottomDivergence = []
TopDivergence = []
extreme_bar_num = np.array(df_only_last.bar_num)
for i in range(len(test)):
    if i >= 138:#需要保证Idx获取到两个值
        Idx = extreme_bar_num[(extreme_bar_num - i) <= 0][-2:]
        first_Idx = Idx[0]
        second_Idx = Idx[1]
        df_sub = test.loc[first_Idx : second_Idx]
        if df_only_last[df_only_last.bar_num == second_Idx].High_or_Low.values[0] == 'ZZPType.LOW_ZZP':#那就要判断是不是底背离
            #获取rsi的最低点的index
            rsi_min = df_sub.RSI.min()
            rsi_Idx = df_sub[df_sub.RSI == rsi_min].index[0]
            #获取价格的最低点的index
            price_min = df_sub.Low.min()
            price_Idx = df_sub[df_sub.Low == price_min].index[0]
            #如果两个index不是同一个，说明背离
            if rsi_Idx != price_Idx:
                BottomDivergence.append(1)
                TopDivergence.append(0)
            else:
                BottomDivergence.append(0)
                TopDivergence.append(0)
        else:#那就要判断是不是顶背离
            rsi_max = df_sub.RSI.max()
            rsi_Idx = df_sub[df_sub.RSI == rsi_max].index[0]
            
            price_max = df_sub.High.max()
            price_Idx = df_sub[df_sub.High == price_max].index[0]
            if rsi_Idx != price_Idx:
                TopDivergence.append(1)
                BottomDivergence.append(0)
            else:
                TopDivergence.append(0)
                BottomDivergence.append(0)
    else:
#         print(i)
        BottomDivergence.append(0)
        TopDivergence.append(0)
        continue
test['BottomDivergence'] = BottomDivergence
test['TopDivergence'] = TopDivergence  

In [23]:
def CompareCurrentRSIwithyes(df):
    df['currentRSI_lessthan_lastRSI'] = [1 if df.iloc[i].RSI < df.iloc[i].previousRSI else 0 for i in range(len(df))]
    df['currentRSI_morethan_lastRSI'] = [1 if df.iloc[i].RSI > df.iloc[i].previousRSI else 0 for i in range(len(df))]

In [24]:
CompareCurrentRSIwithyes(test)

In [25]:
#算Ratio1
def AddRatio1(df1,df2):
    '''
    df1为包含所有擦去的点的表，df2为最终留下的表
    '''

    high_or_low_barnum = np.array(df1[abs(df1.length) >1e-6].bar_num)
    startNum = [0] + high_or_low_barnum[:]
    endNum = high_or_low_barnum[:] + [len(df1)]
    ratio1 = []
    for num in df1.bar_num.tolist():
        if num >= 413:#保证df2中有足够的8个数值可以计算\n", 445
            #print(num)
            
            Previous_extreme_point = high_or_low_barnum[(high_or_low_barnum - num) <= 0][-1]#获取到上一个极值点的barnum
            
            
            if num == Previous_extreme_point:
                Idx = df2[df2.bar_num == num].index[0]
                #print(Idx)
                last5 = []
                count = 0
                while count <= 4:
                    last5.append(abs(df2.loc[Idx - 2*count].length))
                    count += 1
                last5_mean = np.mean(last5)
                last5_std = np.std(last5)
                ratio1_sub = (abs(df2.loc[Idx].length) - last5_mean)/last5_std
                ratio1.append(ratio1_sub)

            else:
                Idx = df2[df2.bar_num == Previous_extreme_point].index[0]
                current_Idx = df1[df1.bar_num == num].index[0]
                first_last5 = abs(df2.loc[Idx].extremum - df1.loc[current_Idx].extremum)
                last5 = [first_last5]
                count = 0
                while count <= 3:
                    last5.append(abs(df2.loc[Idx - 2*count].length))
                    count += 1
                last5_mean = np.mean(last5)
                last5_std = np.std(last5)
                ratio1_sub = (first_last5 - last5_mean)/last5_std
                ratio1.append(ratio1_sub)

                
        else:
            ratio1.append(0)
            continue
    
    print(len(ratio1))
    print(len(df1.bar_num.tolist()))
    df1['ratio1'] = ratio1

In [26]:
AddRatio1(df_his_listed,df_only_last)

371
371


In [27]:
#算Ratio2
#获取了有出现两次的bar，也即这个bar_num为最终未被擦除的\n",
def AddRatio2(df1,df2):
    high_or_low_barnum = np.array(df1[abs(df1.length) >1e-6].bar_num)
    startNum = [0] + high_or_low_barnum[:]
    endNum = high_or_low_barnum[:] + [len(df1)]
    ratio2 = []
    for num in df1.bar_num.tolist():
        if num >= 413:#保证df2中有足够的8个数值可以计算\n", 445
            Previous_extreme_point = high_or_low_barnum[(high_or_low_barnum - num) <= 0][-1]#获取到上一个极值点的barnum
            if num == Previous_extreme_point:
                Idx = df2[df2.bar_num == num].index[0]
                #print(Idx)
                last9 = []
                count = 0
                while count <= 8:
                    last9.append(abs(df2.loc[Idx - count].length))
                    count += 1
                last9_mean = np.mean(last9)
                last9_std = np.std(last9)
                ratio2_sub = (abs(df2.loc[Idx].length) - last9_mean)/last9_std
                ratio2.append(ratio2_sub)
            else:
                Idx = df2[df2.bar_num == Previous_extreme_point].index[0]
                current_Idx = df1[df1.bar_num == num].index[0]
                first_last9 = abs(df2.loc[Idx].extremum - df1.loc[current_Idx].extremum)
                last9 = [first_last9]
                count = 0
                while count <= 7:
                    last9.append(abs(df2.loc[Idx - count].length))
                    count += 1
                last9_mean = np.mean(last9)
                last9_std = np.std(last9)
                ratio2_sub = (first_last9 - last9_mean)/last9_std
                ratio2.append(ratio2_sub)


        else:
            ratio2.append(0)
            continue
    df1['ratio2'] = ratio2
    print(len(ratio2))
    print(len(df1.bar_num.tolist()))

In [28]:
AddRatio2(df_his_listed,df_only_last)

371
371


In [29]:
def CombineRSIandIn(df):

    df['RSIlessthan'] = df.RSI * df.currentRSI_lessthan_lastRSI
    df['RSImorethan'] = df.RSI * df.currentRSI_morethan_lastRSI

In [30]:
CombineRSIandIn(test)

In [31]:
#创建添加dataframe记录的函数
def FindData(df,df1,Idx,Idx_):
    '''
    df为包含indicator基本信息的表，df1为包含zigzag信息的表，Idx为希望获取的记录在df中的index,Idx_大部分时候和Idx相等，但是当Idx值出现两次时，Idx_要往后取
    '''
    
    # gmt = df.loc[Idx].GMT
    time = df.loc[Idx_].Time
    wave5 = df1[df1.bar_num == Idx].iloc[0].ratio1
    wave9 = df1[df1.bar_num == Idx].iloc[0].ratio2
    downpinbar = df.loc[Idx_].DownPinBar
    uppinbar = df.loc[Idx_].UpPinBar
    toptype = df.loc[Idx_].TopType
    bottomtype = df.loc[Idx_].BottomType
    downpregnanttype = df.loc[Idx_].DownPregnant
    uppregnanttype = df.loc[Idx_].UpPregnant
    downswallowtype = df.loc[Idx_].DownSwallowType
    upswallowtype = df.loc[Idx_].UpSwallowType
    RSIlessthan = df.loc[Idx_].RSIlessthan
    RSImorethan = df.loc[Idx_].RSImorethan
    topdivergence = df.loc[Idx_].TopDivergence
    bottomdivergence = df.loc[Idx_].BottomDivergence
    RSIShortgodown = df.loc[Idx_].RSIShortgodown
    RSIShortgoup = df.loc[Idx_].RSIShortgoup
    temp = df1[df1.bar_num == Idx]
    if len(temp) >= 2:
        if temp.iloc[1].High_or_Low == "ZZPType.HIGH_ZZP":
            highorlow = -1
        else:
            highorlow = 1
    else:
        highorlow = 0

    result = {'time':time,'highorlow':highorlow,'wave5':wave5,'wave9':wave9,'downpinbar':downpinbar,'uppinbar':uppinbar,'toptype':toptype,'bottomtype':bottomtype,'downpregnanttype':downpregnanttype,'uppregnanttype':uppregnanttype,'downswallowtype':downswallowtype,'upswallowtype':upswallowtype,'RSIlessthan':RSIlessthan,'RSImorethan':RSImorethan,'topdivergence':topdivergence,'bottomdivergence':bottomdivergence,'RSIShortgodown':RSIShortgodown,'RSIShortgoup':RSIShortgoup}

    return result

In [32]:
def MakeRegressionTable(df,df1):#以zigzag的表为基准
    df_regression = pd.DataFrame(columns = ['time','highorlow','wave5','wave9','downpinbar','uppinbar','toptype','bottomtype','downpregnanttype','uppregnanttype','downswallowtype','upswallowtype','RSIlessthan','RSImorethan','topdivergence','bottomdivergence','RSIShortgodown','RSIShortgoup'])
    for num in df1.bar_num.values.tolist():
        '''
        代码优化，如果zigzag的两个高低点之间的距离太近，赋值highorlow的时候会有冲突，解决办法，以后面那个为准
        新加入的训练集，历史赋值点后3根的y值也令其为0

        '''

        #print(num)

#         Idx = num - 62#获取在df中的index
        Idx = num
        result = FindData(df,df1,Idx,Idx)
        #print(result)
        df_regression = df_regression.append(result,ignore_index=True)
        #isfinaltype = df1[df1.bar_num == num].iloc[0].High_or_Low
#         if len(df1[df1.bar_num == num]) >= 2:
        count = 1
        Idx_ = Idx
        while (count <= 3) & (Idx_ + count < len(df)):
            Idx_ = Idx + count
            result1 = FindData(df,df1,Idx,Idx_)
            #print(result1)
            df_regression = df_regression.append(result1,ignore_index=True)
            count += 1
#         else:
#             continue
    return df_regression    

In [33]:
df_regression = MakeRegressionTable(test,df_his_listed)

In [34]:
df_regression_cleaned = df_regression.drop_duplicates(subset= ['time'], keep='last', inplace=False)

In [35]:
df_regression_cleaned['RSIlessthan'] = df_regression_cleaned['RSIlessthan']/100
df_regression_cleaned['RSImorethan'] = df_regression_cleaned['RSImorethan']/100

<ipython-input-35-cbfa0b11362a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regression_cleaned['RSIlessthan'] = df_regression_cleaned['RSIlessthan']/100
<ipython-input-35-cbfa0b11362a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regression_cleaned['RSImorethan'] = df_regression_cleaned['RSImorethan']/100


In [36]:
df_combine = pd.DataFrame()

In [37]:
df_combine['time'] = test.Time

In [38]:
table = pd.merge(df_combine, df_regression_cleaned, how='left', on=['time'])

In [39]:
df_OCLH_and_Time = test.loc[:,['Time','Open','Close','Low','High','Volume']]
all_data = pd.concat([df_OCLH_and_Time,table.iloc[:,1:]],axis = 1)

In [40]:
# from pyecharts.globals import CurrentConfig,NotebookType,OnlineHostType
# # CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
# CurrentConfig.ONLINE_HOST = "C:/Users/Administrator/Desktop/pythonHistoricalTesting/"

In [41]:
from typing import List,Sequence,Union
from pyecharts import options as opts 
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Kline,Line,Bar,Grid
from pyecharts.globals import ThemeType

In [42]:
echarts_data = all_data.values.tolist()

In [43]:
all_data

,Time,Open,Close,Low,High,Volume,highorlow,wave5,wave9,downpinbar,...,downpregnanttype,uppregnanttype,downswallowtype,upswallowtype,RSIlessthan,RSImorethan,topdivergence,bottomdivergence,RSIShortgodown,RSIShortgoup
0,2020-12-06 17:00:00,30174.0,30218.0,30171.0,30265.0,1364.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-06 17:15:00,30220.0,30247.0,30205.0,30247.0,750.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-12-06 17:30:00,30248.0,30248.0,30224.0,30263.0,669.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-06 17:45:00,30248.0,30225.0,30219.0,30261.0,406.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-12-06 18:00:00,30225.0,30202.0,30159.0,30228.0,1855.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,2021-01-05 05:30:00,30179.0,30141.0,30127.0,30179.0,1051.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1766,2021-01-05 05:45:00,30143.0,30122.0,30098.0,30148.0,890.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1767,2021-01-05 06:00:00,30123.0,30095.0,30066.0,30136.0,1545.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1768,2021-01-05 06:15:00,30094.0,30034.0,30030.0,30094.0,1543.0,0,-1.175941,-0.811473,0,...,0,0,0,0,0.37647,0.000000,1,0,0,0


In [44]:
all_data.fillna(3)

,Time,Open,Close,Low,High,Volume,highorlow,wave5,wave9,downpinbar,...,downpregnanttype,uppregnanttype,downswallowtype,upswallowtype,RSIlessthan,RSImorethan,topdivergence,bottomdivergence,RSIShortgodown,RSIShortgoup
0,2020-12-06 17:00:00,30174.0,30218.0,30171.0,30265.0,1364.0,3,3.000000,3.000000,3,...,3,3,3,3,3.00000,3.000000,3,3,3,3
1,2020-12-06 17:15:00,30220.0,30247.0,30205.0,30247.0,750.0,3,3.000000,3.000000,3,...,3,3,3,3,3.00000,3.000000,3,3,3,3
2,2020-12-06 17:30:00,30248.0,30248.0,30224.0,30263.0,669.0,3,3.000000,3.000000,3,...,3,3,3,3,3.00000,3.000000,3,3,3,3
3,2020-12-06 17:45:00,30248.0,30225.0,30219.0,30261.0,406.0,3,3.000000,3.000000,3,...,3,3,3,3,3.00000,3.000000,3,3,3,3
4,2020-12-06 18:00:00,30225.0,30202.0,30159.0,30228.0,1855.0,3,3.000000,3.000000,3,...,3,3,3,3,3.00000,3.000000,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1765,2021-01-05 05:30:00,30179.0,30141.0,30127.0,30179.0,1051.0,3,3.000000,3.000000,3,...,3,3,3,3,3.00000,3.000000,3,3,3,3
1766,2021-01-05 05:45:00,30143.0,30122.0,30098.0,30148.0,890.0,3,3.000000,3.000000,3,...,3,3,3,3,3.00000,3.000000,3,3,3,3
1767,2021-01-05 06:00:00,30123.0,30095.0,30066.0,30136.0,1545.0,3,3.000000,3.000000,3,...,3,3,3,3,3.00000,3.000000,3,3,3,3
1768,2021-01-05 06:15:00,30094.0,30034.0,30030.0,30094.0,1543.0,0,-1.175941,-0.811473,0,...,0,0,0,0,0.37647,0.000000,1,0,0,0


In [45]:
all_data.to_csv('./code_generated_csv/all_data.csv')

In [46]:
def split_data(origin_data)->dict:
    datas = []
    times = []
    vols = []
    highorlow = []
    wave5 = []
    wave9 = []
    downpinbar = []
    uppinbar = []
    toptype = []
    bottomtype = []
    downpregnanttype = []
    uppregnanttype = []
    downswallowtype = []
    upswallowtype = []
    RSIlessthan = []
    RSImorethan = []
    topdivergence = []
    bottomdivergence = []
    RSIShortgodown = []
    RSIShortgoup = []

    for i in range(len(origin_data)):
        datas.append(origin_data[i][1:5])
        times.append((origin_data[i][0]).strftime('%Y-%m-%d %H:%M:%S'))
        vols.append(origin_data[i][5])
        highorlow.append(origin_data[i][6])
        wave5.append(origin_data[i][7])
        wave9.append(origin_data[i][8])
        downpinbar.append(origin_data[i][9])
        uppinbar.append(origin_data[i][10])
        toptype.append(origin_data[i][11])
        bottomtype.append(origin_data[i][12])
        downpregnanttype.append(origin_data[i][13])
        uppregnanttype.append(origin_data[i][14])
        downswallowtype.append(origin_data[i][15])
        upswallowtype.append(origin_data[i][16])
        RSIlessthan.append(origin_data[i][17])
        RSImorethan.append(origin_data[i][18])
        topdivergence.append(origin_data[i][19])
        bottomdivergence.append(origin_data[i][20])
        RSIShortgodown.append(origin_data[i][21])
        RSIShortgoup.append(origin_data[i][22])
    vols = [int(v) for v in vols]

    return {
        "datas":datas,
        "times":times,
        "vols":vols,
        "highorlow":highorlow,
        "wave5":wave5,
        "wave9":wave9,
        "downpinbar":downpinbar,
        "uppinbar":uppinbar,
        "toptype":toptype,
        "bottomtype":bottomtype,
        "downpregnanttype":downpregnanttype,
        "uppregnanttype":uppregnanttype,
        "downswallowtype":downswallowtype,
        "upswallowtype":upswallowtype,
        "RSIlessthan":RSIlessthan,
        "RSImorethan":RSImorethan,
        "topdivergence":topdivergence,
        "bottomdivergence":bottomdivergence,
        "RSIShortgodown":RSIShortgodown,
        "RSIShortgoup":RSIShortgoup 
        
    }

In [47]:
data = split_data(echarts_data)

In [48]:
# def split_data_part() -> Sequence:
# 	mark_line_data = []
# 	idx = 0
# 	tag = 0
# 	vols = 0
# 	for i in range(len(data["times"])):
# 		if data["datas"][i][5] != 0 and tag == 0:
# 			idx = i
# 			vols = data["datas"][i][4]
# 			tag = 1
# 		if tag == 1:
# 			vols += data["datas"][i][4]
# 		if data["datas"][i][5] != 0 or tag == 1:
# 			mark_line_data.append(
# 				[
# 					{
# 						"xAxis": idx,
# 						"yAxis": float("%.2f" % data["datas"][idx][3])
# 						if data["datas"][idx][1] > data["datas"][idx][0]
# 						else float("%.2f" % data["datas"][idx][2]),
# 						"value": vols,
# 					},
# 					{
# 						"xAxis": i,
# 						"yAxis": float("%.2f" % data["datas"][i][3])
# 						if data["datas"][i][1] > data["datas"][i][0]
# 						else float("%.2f" % data["datas"][i][2]),
# 					},
# 				]
# 			)
# 			idx = i
# 			vols = data["datas"][i][4]
# 			tag = 2
# 		if tag == 2:
# 			vols += data["datas"][i][4]
# 		if data["datas"][i][5] != 0 and tag == 2:
# 			mark_line_data.append(
# 				[
# 					{
# 						"xAxis": idx,
# 						"yAxis": float("%.2f" % data["datas"][idx][3])
# 						if data["datas"][i][1] > data["datas"][i][0]
# 						else float("%.2f" % data["datas"][i][2]),
# 						"value": str(float("%.2f" % (vols / (i - idx + 1)))) + " M",
# 					},
# 					{
# 						"xAxis": i,
# 						"yAxis": float("%.2f" % data["datas"][i][3])
# 						if data["datas"][i][1] > data["datas"][i][0]
# 						else float("%.2f" % data["datas"][i][2]),
# 					},
# 				]
# 			)
# 			idx = i
# 			vols = data["datas"][i][4]
# 	print(mark_line_data)
# 	return mark_line_data

In [49]:
def calculate_ma(day_count: int):
	result: List[Union[float, str]] = []

	for i in range(len(data["times"])):
		if i < day_count:
			result.append("-")
			continue
		sum_total = 0.0
		for j in range(day_count):
			sum_total += float(data["datas"][i - j][1])
		result.append(abs(float("%.2f" % (sum_total / day_count))))
	return result

In [50]:
all_data.columns

Index(['Time', 'Open', 'Close', 'Low', 'High', 'Volume', 'highorlow', 'wave5',
       'wave9', 'downpinbar', 'uppinbar', 'toptype', 'bottomtype',
       'downpregnanttype', 'uppregnanttype', 'downswallowtype',
       'upswallowtype', 'RSIlessthan', 'RSImorethan', 'topdivergence',
       'bottomdivergence', 'RSIShortgodown', 'RSIShortgoup'],
      dtype='object')

In [57]:
kline = (
        Kline()
        .add_xaxis(xaxis_data = data["times"])
        .add_yaxis('',y_axis = data['datas'])
        .set_global_opts(
            xaxis_opts = opts.AxisOpts(
                type_ = 'category',
                is_scale = True,
                boundary_gap = False,
                axisline_opts = opts.AxisLineOpts(is_on_zero = False),
                splitline_opts = opts.SplitLineOpts(is_show = False),
                split_number = 20,
                min_ = 'dataMin',
                max_ = 'dataMax'),
            yaxis_opts = opts.AxisOpts(
                is_scale = True,
                splitline_opts = opts.SplitLineOpts(is_show = True)),
            legend_opts = opts.LegendOpts(pos_left = '3%',orient = 'vertical'),
#             grid_index = 0,###
            toolbox_opts = opts.ToolboxOpts(is_show = True,pos_top = 'top'),
            datazoom_opts = 
            [opts.DataZoomOpts(is_show = True,
                               type_ = 'slider',
                               is_realtime = True,
                               orient = "horizontal",
                               range_start = 0,range_end = 100,
                               xaxis_index = [0,0]),
             opts.DataZoomOpts(is_show = True,
                               range_start = 0,range_end = 100,
                               xaxis_index = [0,1]),
             opts.DataZoomOpts(is_show = True,
                               range_start = 0,range_end = 100,
                               xaxis_index = [0,2]),
             opts.DataZoomOpts(is_show = True,
                               range_start = 0,range_end = 100,
                               xaxis_index = [0,3]),
            ]
        )
)

kline_line = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "MA5",
        y_axis = calculate_ma(day_count = 5),
        is_smooth = True,
        linestyle_opts = opts.LineStyleOpts(opacity = 0.5),
        label_opts = opts.LabelOpts(is_show = False),
        )
    .set_global_opts(
        xaxis_opts = opts.AxisOpts(
            type_ = "category",
            axislabel_opts = opts.LabelOpts(is_show = False),
        ),
        yaxis_opts = opts.AxisOpts(
            split_number = 3,
            axisline_opts = opts.AxisLineOpts(is_on_zero = False),
            axistick_opts = opts.AxisTickOpts(is_show = False),
            splitline_opts = opts.SplitLineOpts(is_show = False),
            axislabel_opts = opts.LabelOpts(is_show = True),
            ),
        )
    )
overlap_kline_line = kline.overlap(kline_line)
#目前不太需要成交量的图
# bar_1 = (
#     Bar()
#     .add_xaxis(xaxis_data = data["times"])
#     .add_yaxis(
#         series_name = "Volumn",
#         yaxis_data = data["vols"],
#         label_opts = opts.LabelOpts(is_show = False),
#         itemstyle_opts = opts.ItemStyleOpts(
#             color = JsCode(
#                     """
#                     function(params){
#                         var colorList;
#                         if(barData[params.dataIndex][1] > barData[params.dataIndex][0]){
#                             colorList = '#ef232a';
#                         }else{
#                             colorList = '#14b143';
#                         }
#                         return colorList;
#                     }
#                     """
#                 )
#             ),
#         )
#     .set_global_opts(
#         xaxis_opts = opts.AxisOpts(
#             type_ = "category",
#             axislabel_opts = opts.LabelOpts(is_show = False),
#             ),
#         )
#     )

#wave5同向的波
line_1 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "wave5",
        y_axis = data["wave5"],
        label_opts = opts.LabelOpts(is_show = False),
        itemstyle_opts = opts.ItemStyleOpts(
            color = JsCode(
                """
                    function(params){
                        var colorList;
                        if(params.data >= 0){
                            colorList = '#ef232a';
                        }else{
                            colorList = '#14b143';
                        }
                        return colorList;
                    }
                """
                )
            ),
        )
    .set_global_opts(
        xaxis_opts = opts.AxisOpts(
            type_ = "category",
            axislabel_opts = opts.LabelOpts(is_show = False),
            ),
        yaxis_opts = opts.AxisOpts(
            split_number = 4,#####
            axisline_opts = opts.AxisLineOpts(is_on_zero = False),
            axistick_opts = opts.AxisTickOpts(is_show = False),
            splitline_opts = opts.SplitLineOpts(is_show = False),
            axislabel_opts = opts.LabelOpts(is_show = True),
            ),
        legend_opts = opts.LegendOpts(is_show =True,pos_left = '3%',pos_top = '3%',orient = 'vertical'),
        )
    )
#wave9
line_2 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "wave9",
        y_axis = data["wave9"],
        label_opts = opts.LabelOpts(is_show = False),
        itemstyle_opts = opts.ItemStyleOpts(
            color = JsCode(
                """
                    function(params){
                        var colorList;
                        if(params.data >= 0){
                            colorList = '#ef232a';
                        }else{
                            colorList = '#14b143';
                        }
                        return colorList;
                    }
                """
                )
            ),
        )
    .set_global_opts(
        xaxis_opts = opts.AxisOpts(
            type_ = "category",
            axislabel_opts = opts.LabelOpts(is_show = False),
            ),
        yaxis_opts = opts.AxisOpts(
            split_number = 4,#####
            axisline_opts = opts.AxisLineOpts(is_on_zero = False),
            axistick_opts = opts.AxisTickOpts(is_show = False),
            splitline_opts = opts.SplitLineOpts(is_show = False),
            axislabel_opts = opts.LabelOpts(is_show = True),
            ),
        legend_opts = opts.LegendOpts(is_show =True),
        )
    )
overlap_line1_line2 = line_1.overlap(line_2)


#downpinbar
line_3 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "downpinbar",
        y_axis = data["downpinbar"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True,pos_left = '3%',pos_top = '10%',orient = 'vertical'),
    
    )
)
#uppinbar
line_4 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "uppinbar",
        y_axis = data["uppinbar"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line3_line4 = line_3.overlap(line_4)
#toptype
line_5 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "toptype",
        y_axis = data["toptype"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line4_line5 = overlap_line3_line4.overlap(line_5)   
 #bottomtype
line_6 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "bottomtype",
        y_axis = data["bottomtype"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line5_line6 = overlap_line4_line5.overlap(line_6) 
#downpregnanttype
line_7 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "downpregnanttype",
        y_axis = data["downpregnanttype"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line6_line7 = overlap_line5_line6.overlap(line_7)
#uppregnanttype
line_8 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "uppregnanttype",
        y_axis = data["uppregnanttype"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line7_line8 = overlap_line6_line7.overlap(line_8)
#downswallowtype 
line_9 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "downswallowtype",
        y_axis = data["downswallowtype"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line8_line9 = overlap_line7_line8.overlap(line_9)
#upswallowtype
line_10 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "upswallowtype",
        y_axis = data["upswallowtype"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line9_line10 = overlap_line8_line9.overlap(line_10)
#topdivergence
line_11 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "topdivergence",
        y_axis = data["topdivergence"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line10_line11 = overlap_line9_line10.overlap(line_11)
#bottomdivergence
line_12 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "bottomdivergence",
        y_axis = data["bottomdivergence"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line11_line12 = overlap_line10_line11.overlap(line_12)


#RSIlessthan
line_13 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "RSIlessthan",
        y_axis = data["RSIlessthan"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True,pos_left = '3%',pos_top = '40%',orient = 'vertical'),
    )
)
#RSImorethan
line_14 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "RSImorethan",
        y_axis = data["RSImorethan"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line13_line14 = line_13.overlap(line_14)
 #RSIShortgodown
line_15 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "RSIShortgodown",
        y_axis = data["RSIShortgodown"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line14_line15 = overlap_line13_line14.overlap(line_15)
#RSIShortgoup
line_16 = (
    Line()
    .add_xaxis(xaxis_data = data["times"])
    .add_yaxis(
        series_name = "RSIShortgoup",
        y_axis = data["RSIShortgoup"],
        label_opts = opts.LabelOpts(is_show =False),
        )
    .set_global_opts(
    xaxis_opts = opts.AxisOpts(
        type_ = "category",
        axislabel_opts = opts.LabelOpts(is_show = False),
        ),
    yaxis_opts = opts.AxisOpts(
        split_number = 4,#####
        axisline_opts = opts.AxisLineOpts(is_on_zero = False),
        axistick_opts = opts.AxisTickOpts(is_show = False),
        splitline_opts = opts.SplitLineOpts(is_show = False),
        axislabel_opts = opts.LabelOpts(is_show = True),
        ),
    legend_opts = opts.LegendOpts(is_show =True),
    )
)
overlap_line15_line16 = overlap_line14_line15.overlap(line_16)





grid_chart = Grid(init_opts = opts.InitOpts(width = "1400px",height = "800px",theme=ThemeType.VINTAGE))#,theme=ThemeType.PURPLE_PASSION
grid_chart.add_js_funcs("var barData={}".format(data["datas"]))            
grid_chart.add(
    overlap_kline_line,
    grid_opts = opts.GridOpts(pos_left = '18%',pos_top = 'top',height = '30%'))#grid_index = 0,
grid_chart.add(
    overlap_line1_line2,
    grid_opts = opts.GridOpts(pos_left = '18%',pos_top = '35%',height = '20%'))#grid_index = 0,
grid_chart.add(
    overlap_line11_line12,
    grid_opts = opts.GridOpts(pos_left = '18%',pos_top = '55%',height = '20%'))#grid_index = 0,
grid_chart.add(
    overlap_line15_line16,
    grid_opts = opts.GridOpts(pos_left = '18%',pos_top = '75%',height = '20%'))#grid_index = 0,
grid_chart.render('test_kline_chart.html')

'C:\\Users\\Administrator\\Desktop\\pythonHistoricalTesting\\test_kline_chart.html'

In [52]:
# Draw()

In [53]:
# def draw_kline():
# 	kline = (
# 		Kline()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "",
# 			y_axis = data["datas"],#可能是这边的datas有点问题
# 			itemstyle_opts = opts.ItemStyleOpts(
# 				color = "#ef232a",
# 				color0 = "#14b143",
# 				border_color = "#ef232a",
# 				border_color0 = "#14b143"
# 				),
# 		)
# 		.set_global_opts(
# 			xaxis_opts = opts.AxisOpts(
# 				type_ = "category",
# 				is_scale = True,
# 				boundary_gap = False,
# 				axisline_opts = opts.AxisLineOpts(is_on_zero = False),
# 				splitline_opts = opts.SplitLineOpts(is_show = False),
# 				split_number = 20,
# 				min_ = "dataMin",
# 				max_ = "dataMax",
# 				),
# 			yaxis_opts = opts.AxisOpts(
# 				is_scale = True,
# 				splitline_opts = opts.SplitLineOpts(is_show = True)
# 				),
# 			datazoom_opts = [
# 				opts.DataZoomOpts(
# 					is_show = True,type_ = "slider",xaxis_index = [0,0],range_end = 100
# 					),
# 			],
# 		)
# 	)
# 	grid_chart = Grid(init_opts = opts.InitOpts(width = "1400px",height = "800px",theme = ThemeType.ESSOS))
# 	grid_chart.add_js_funcs("var barData={}".format(data["datas"]))
# 	grid_chart.add(
# 		kline,
# 		grid_opts = opts.GridOpts(
# 			pos_left="3%", pos_right="1%", height="60%"
# 			),
# 		)
# 	grid_chart.render("test_kline_chart.html")

In [54]:
# draw_kline()

In [55]:
# def draw_chart():
# 	kline = (
# 		Kline()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "",
# 			y_axis = data["datas"],#可能是这边的datas有点问题
# 			itemstyle_opts = opts.ItemStyleOpts(
# 				color = "#ef232a",
# 				color0 = "#14b143",
# 				border_color = "#ef232a",
# 				border_color0 = "#14b143"
# 				),
# # 			markpoint_opts = opts.MarkPointOpts(
# # 				data = [
# # 					opts.MarkPointItem(type_ = "max",name = "最大值"),
# # 					opts.MarkPointItem(type_ = "min",name = "最小值"),
# # 				]
# # 			),
# # 			markline_opts = opts.MarkLineOpts(
# # 				label_opts = opts.LabelOpts(
# # 					position = "middle",
# # 					color = "blue",
# # 					font_size = 15
# # 					),
# # 				data = split_data_part(),
# # 				symbol = ["circle","none"],
# # 				),
# 			)
# 		.set_series_opts(
# 			markarea_opts = opts.MarkAreaOpts(is_silent = True,data = split_data_part())
# 			)
# 		.set_global_opts(
# 			title_opts = opts.TitleOpts(title = "K线周期图表",pos_left = "0"),
# 			xaxis_opts = opts.AxisOpts(
# 				type_ = "category",
# 				is_scale = True,
# 				boundary_gap = False,
# 				axisline_opts = opts.AxisLineOpts(is_on_zero = False),
# 				splitline_opts = opts.SplitLineOpts(is_show = False),
# 				split_number = 20,
# 				min_ = "dataMin",
# 				max_ = "dataMax",
# 				),
# 			yaxis_opts = opts.AxisOpts(
# 				is_scale = True,
# 				splitline_opts = opts.SplitLineOpts(is_show = True)
# 				),
# 			tooltip_opts = opts.TooltipOpts(trigger = "axis",axis_pointer_type = "line"),
# 			datazoom_opts = [
# 				opts.DataZoomOpts(
# 					is_show = True,type_ = "inside",xaxis_index = [0,0],range_end = 100
# 					),
# 				opts.DataZoomOpts(
# 					is_show = True,xaxis_index = [0,1],pos_top = "97%",range_end = 100
# 					),
# 				opts.DataZoomOpts(is_show = False,xaxis_index = [0,2],range_end = 100),
# # 				opts.DataZoomOpts(is_show = True,xaxis_index = [0,3],range_end = 100),
# # 				opts.DataZoomOpts(is_show = True,xaxis_index = [0,4],range_end = 100),
# # 				opts.DataZoomOpts(is_show = True,xaxis_index = [0,5],range_end = 100),
# # 				opts.DataZoomOpts(is_show = True,xaxis_index = [0,6],range_end = 100),
# # 				opts.DataZoomOpts(is_show = True,xaxis_index = [0,7],range_end = 100),
# # 				opts.DataZoomOpts(is_show = False,xaxis_index = [0,8],range_end = 100),
# 			],
# 			)
# 		)
# # 	kline.load_javascript() 
# 	kline_line = (
# 			Line()
# 			.add_xaxis(xaxis_data = data["times"])
# 			.add_yaxis(
# 				series_name = "MA5",
# 				y_axis = calculate_ma(day_count = 5),
# 				is_smooth = True,
# 				linestyle_opts = opts.LineStyleOpts(opacity = 0.5),
# 				label_opts = opts.LabelOpts(is_show = False),
# 				)
# 			.set_global_opts(
# 				xaxis_opts = opts.AxisOpts(
# 					type_ = "category",
# 					axislabel_opts = opts.LabelOpts(is_show = False),
# 				),
# 				yaxis_opts = opts.AxisOpts(
# 					split_number = 3,
# 					axisline_opts = opts.AxisLineOpts(is_on_zero = False),
# 					axistick_opts = opts.AxisTickOpts(is_show = False),
# 					splitline_opts = opts.SplitLineOpts(is_show = False),
# 					axislabel_opts = opts.LabelOpts(is_show = True),
# 					),
# 				)
# 		)
# 	kline_line.load_javascript() 
# 	overlap_kline_line = kline.overlap(kline_line)
# 	overlap_kline_line.load_javascript() 
# #成交量
# 	bar_1 = (
# 		Bar()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "Volumn",
# 			yaxis_data = data["vols"],
# 			label_opts = opts.LabelOpts(is_show = False),
# 			itemstyle_opts = opts.ItemStyleOpts(
# 				color = JsCode(
# 						"""
# 						function(params){
# 							var colorList;
# 							if(barData[params.dataIndex][1] > barData[params.dataIndex][0]){
# 								colorList = '#ef232a';
# 							}else{
# 								colorList = '#14b143';
# 							}
# 							return colorList;
# 						}
# 						"""
# 					)
# 				),
# 			)
# 		.set_global_opts(
# 			xaxis_opts = opts.AxisOpts(
# 				type_ = "category",
# 				axislabel_opts = opts.LabelOpts(is_show = False),
# 				),
# 			legend_opts = opts.LegendOpts(is_show = False),
# 			)
# 		)
# #highorlow
# 	line = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "highorlow",
# 			y_axis = data["highorlow"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_bar1_line = bar_1.overlap(line)  
# #wave5同向的波
# 	bar_2 = (
# 		Bar()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "wave5",
# 			yaxis_data = data["wave5"],
# 			label_opts = opts.LabelOpts(is_show = False),
# 			itemstyle_opts = opts.ItemStyleOpts(
# 				color = JsCode(
# 					"""
# 						function(params){
# 							var colorList;
# 							if(params.data >= 0){
# 								colorList = '#ef232a';
# 							}else{
# 								colorList = '#14b143';
# 							}
# 							return colorList;
# 						}
# 					"""
# 					)
# 				),
# 			)
# 		.set_global_opts(
# 			xaxis_opts = opts.AxisOpts(
# 				type_ = "category",
# 				axislabel_opts = opts.LabelOpts(is_show = False),
# 				),
# 			yaxis_opts = opts.AxisOpts(
# 				split_number = 4,#####
# 				axisline_opts = opts.AxisLineOpts(is_on_zero = False),
# 				axistick_opts = opts.AxisTickOpts(is_show = False),
# 				splitline_opts = opts.SplitLineOpts(is_show = False),
# 				axislabel_opts = opts.LabelOpts(is_show = True),
# 				),
# 			legend_opts = opts.LegendOpts(is_show =False),
# 			)
# 		)
# #wave9
# 	bar_3 = (
# 		Bar()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "wave9",
# 			yaxis_data = data["wave9"],
# 			label_opts = opts.LabelOpts(is_show = False),
# 			itemstyle_opts = opts.ItemStyleOpts(
# 				color = JsCode(
# 					"""
# 						function(params){
# 							var colorList;
# 							if(params.data >= 0){
# 								colorList = '#ef232a';
# 							}else{
# 								colorList = '#14b143';
# 							}
# 							return colorList;
# 						}
# 					"""
# 					)
# 				),
# 			)
# 		.set_global_opts(
# 			xaxis_opts = opts.AxisOpts(
# 				type_ = "category",
# 				axislabel_opts = opts.LabelOpts(is_show = False),
# 				),
# 			yaxis_opts = opts.AxisOpts(
# 				split_number = 4,#####
# 				axisline_opts = opts.AxisLineOpts(is_on_zero = False),
# 				axistick_opts = opts.AxisTickOpts(is_show = False),
# 				splitline_opts = opts.SplitLineOpts(is_show = False),
# 				axislabel_opts = opts.LabelOpts(is_show = True),
# 				),
# 			legend_opts = opts.LegendOpts(is_show =False),
# 			)
# 		)
# #downpinbar
# 	line_1 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "downpinbar",
# 			y_axis = data["downpinbar"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# #uppinbar
# 	line_2 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "uppinbar",
# 			y_axis = data["uppinbar"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line1_line2 = line_1.overlap(line_2)
# #  toptype
# 	line_3 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "toptype",
# 			y_axis = data["toptype"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line12_line3 = overlap_line1_line2.overlap(line_3)    
# #  bottomtype
# 	line_4 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "bottomtype",
# 			y_axis = data["bottomtype"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line123_line4 = overlap_line12_line3.overlap(line_4)
# #downpregnanttype
# 	line_5 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "downpregnanttype",
# 			y_axis = data["downpregnanttype"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line1234_line5 = overlap_line123_line4.overlap(line_5)
# #uppregnanttype
# 	line_6 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "uppregnanttype",
# 			y_axis = data["uppregnanttype"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line12345_line6 = overlap_line1234_line5.overlap(line_6)
# #downswallowtype 
# 	line_7 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "downswallowtype",
# 			y_axis = data["downswallowtype"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# #upswallowtype
# 	line_8 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "upswallowtype",
# 			y_axis = data["upswallowtype"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line7_line8 = line_7.overlap(line_8)
# #topdivergence
# 	line_9 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "topdivergence",
# 			y_axis = data["topdivergence"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line78_line9 = overlap_line7_line8.overlap(line_9)
# #bottomdivergence
# 	line_10 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "bottomdivergence",
# 			y_axis = data["bottomdivergence"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line789_line10 = overlap_line78_line9.overlap(line_10)  
# #RSIlessthan
# 	line_11 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "RSIlessthan",
# 			y_axis = data["RSIlessthan"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# #RSImorethan  
# 	line_12 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "RSImorethan",
# 			y_axis = data["RSImorethan"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line11_line12 = line_11.overlap(line_12)
# #  RSIShortgodown
# 	line_13 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "RSIShortgodown",
# 			y_axis = data["RSIShortgodown"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line1112_line13 = overlap_line11_line12.overlap(line_13)
# #RSIShortgoup
# 	line_14 = (
# 		Line()
# 		.add_xaxis(xaxis_data = data["times"])
# 		.add_yaxis(
# 			series_name = "RSIShortgoup",
# 			y_axis = data["RSIShortgoup"],
# 			label_opts = opts.LabelOpts(is_show =False),
# 			)
# 		.set_global_opts(
# 			legend_opts = opts.LegendOpts(is_show = False)
# 			)
# 		)
# 	overlap_line111213_line14 = overlap_line1112_line13.overlap(line_14) 
# 	grid_chart = Grid(init_opts = opts.InitOpts(width = "1400px",height = "800px",theme = ThemeType.ESSOS))
# 	grid_chart.add_js_funcs("var barData={}".format(data["datas"]))
# 	grid_chart.add(
# 		overlap_kline_line,
# 		grid_opts = opts.GridOpts(
# 			pos_left="3%", pos_right="1%", height="60%"
# 			),
# 		)
# 	grid_chart.add(
# 		overlap_bar1_line,
# 		grid_opts = opts.GridOpts(
# 			pos_left = "3%",pos_right = "1%",pos_top = "71%",height = "10%"
# 			),
# 		)
# 	grid_chart.add(
# 		bar_2,
# 		grid_opts = opts.GridOpts(
# 			pos_left = "3%",pos_right = "1%",pos_top = "71%",height = "10%"
# 			),
# 		)
# 	grid_chart.add(
# 		bar_3,
# 		grid_opts = opts.GridOpts(
# 			pos_left = "3%",pos_right = "1%",pos_top = "71%",height = "10%"
# 			),
# 		)  
# 	grid_chart.add(
# 		overlap_line12345_line6,
# 		grid_opts = opts.GridOpts(
# 			pos_left = "3%",pos_right = "1%",pos_top = "82%",height = "14%"
# 			)
# 		) 
# 	grid_chart.add(
# 		overlap_line789_line10,
# 		grid_opts = opts.GridOpts(
# 			pos_left = "3%",pos_right = "1%",pos_top = "82%",height = "14%"
# 			)
# 		)
# 	grid_chart.add(
# 		overlap_line111213_line14,
# 		grid_opts = opts.GridOpts(
# 			pos_left = "3%",pos_right = "1%",pos_top = "82%",height = "14%"
# 			)
# 		)
# 	grid_chart.render("test_kline_chart.html")

In [56]:
# draw_chart()